In [1]:
!unzip data.zip


Archive:  data.zip
replace data/test.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/text_prepare_tests.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace data/train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
import sys
sys.path.append("..")
import nltk 
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from ast import literal_eval
import pandas as pd
import numpy as np

In [3]:
def read_data(filenmae):
    data=pd.read_csv(filenmae,sep='\t')
    data['tags']=data['tags'].apply(literal_eval)
    return data

In [4]:
train=read_data('data/train.tsv')
validation=read_data('data/validation.tsv')
test=pd.read_csv('data/test.tsv',sep='\t')

In [5]:
from nltk.tokenize import word_tokenize
from string import punctuation

stop_words=set(stopwords.words('english'))
table = str.maketrans('','',punctuation)

In [6]:
def textclean(text):
    tokens=word_tokenize(text)
    tokens=[word for word in tokens if word.isalpha()]
    tokens=[w.translate(table) for w in tokens]
    tokens=[word for word in tokens if not word in stop_words]
    token = [word for word in tokens if len(word)>1 ]
    return tokens

In [7]:
nltk.download('punkt')
contents_train=[]
contents_validation=[]
for index,row in train.iterrows():
    text=row['title'].lower()
    contents_train.append(textclean(text))
    
for index,row in validation.iterrows():
    text=row['title'].lower()
    contents_validation.append(textclean(text))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abdo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
import gensim
from gensim.models import Word2Vec
n_dim=50

w2v_model_train =Word2Vec(contents_train,min_count=5,size=n_dim)
w2v_model_validation =Word2Vec(contents_validation,min_count=5,size=n_dim)

D:\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
 #from sklearn.feature_extraction.text import TfidfVectorizer


#vectorizer  = TfidfVectorizer(min_df=10,analyzer=lambda x: x)

#matrix=vectorizer.fit_transform(contents)
#tfidf = dict(zip(vectorizer.get_feature_names(),vectorizer.idf_))


In [10]:
def create_word_vector(l,size,flag):
    vector = np.zeros(size).reshape((1,size))
    count = 0.
    for word in l:
        try:
            if flag == 'train':
                vector += w2v_model_train[word].reshape((1, size)) 
                count+=1
            else:
                vector += w2v_model_validation[word].reshape((1, size)) 
                count+=1
        except KeyError:
            continue
            
    if count!=0:
        vector /= count
    return vector

X_train = []
y_train = []
X_validation = []
y_validation = []
for i in range(len(train)):
    converted_review = create_word_vector(contents_train[i],n_dim,'train')
    #print(converted_review)
    X_train.append(np.array(converted_review))
    y_train.append(train['tags'][i]) 
for i in range(len(validation)):
    converted_review = create_word_vector(contents_validation[i],n_dim,'validation')
    #print(converted_review)
    X_validation.append(np.array(converted_review))
    y_validation.append(validation['tags'][i]) 

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [12]:
from sklearn.preprocessing import scale

X_train = np.concatenate(X_train)
X_train = scale(X_train)
y_train = np.array(y_train)


X_validation = np.concatenate(X_validation)
X_validation = scale(X_validation)
y_validation = np.array(y_validation)


In [13]:
X_train=X_train[:200,]
y_train=y_train[:200,]

X_validation=X_validation[:200,]
y_validation=y_validation[:200,]



(200, 50)

In [27]:
X_validation.shape

(200, 50)

In [14]:
from collections import Counter
tags_counts=Counter([item for taglist in y_train for item in taglist])

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
y_train = mlb.fit_transform(y_train)
y_validation = mlb.fit_transform(y_validation)

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:935: UserWarning: unknown class(es) ['apache', 'cocoa-touch', 'css', 'datetime', 'maven', 'node.js', 'oop', 'pointers', 'qt', 'servlets', 'session', 'sorting', 'sql-server', 'windows'] will be ignored
  .format(sorted(unknown, key=str)))


In [16]:
y_train[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [24]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
clf = OneVsRestClassifier(SVC())
clf = clf.fit(X_train,y_train)


D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 

In [28]:
y_val_predicted = clf.predict(X_validation)
#y_val_predicted = clf.decision_function(X_validation)

In [33]:
y_val_predicted[100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [34]:

y_val_inversed = mlb.inverse_transform(y_validation)
y_val_pred_inversed = mlb.inverse_transform(y_val_predicted)

for i in range(3):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_validation[i],
        ','.join(y_val_inversed[i]),
        ','.join(y_val_pred_inversed[i])
    ))

Title:	[ 0.11679117 -0.23376937  0.32017152  0.25195818 -0.21679823  0.25522056
  0.14753756 -0.22734404  0.11462275 -0.19095152 -0.31169966 -0.16814315
  0.03166818  0.13763481 -0.14175447 -0.17827859 -0.19693913  0.11973203
 -0.29071804  0.5318933  -0.07167161  0.08576058  0.37884335 -0.17231776
 -0.10860363  0.34962239 -0.02605642 -0.18682012  0.24875638 -0.23045735
 -0.10648938 -0.17986901  0.5512759   0.33767027 -0.42399659  0.32456116
 -0.36257213 -0.02800825 -0.13865849 -0.03192846 -0.54119241  0.18689579
 -0.13281453 -0.28407367  0.11329318  0.22477772  0.18481944  0.07984482
 -0.13581252  0.35901323]
True labels:	php,sql
Predicted labels:	


Title:	[-0.79113664 -0.33733555 -0.51852475  1.33028542 -0.69027663  0.2767712
 -0.21881877 -1.29279454 -0.28135887  0.92449406 -1.27155522  0.63733805
  1.60569526  1.00669699 -0.38546563 -1.11903359 -0.84097626 -1.32187273
 -0.08441174  1.35901869  0.47447859 -1.20438086 -0.11374409 -0.80529008
  0.65232141 -0.14813751  0.93160582  1.466

In [0]:
y_val_predicted[2]

array([-0.9815713 , -1.07776273, -1.07940795, -1.40822688, -1.21296406,
       -1.17409144, -1.12819125, -0.98295461, -1.12501035, -1.03503595,
       -1.14560002, -1.04939606, -1.04777579, -1.01920699, -1.75661003,
       -1.03484748, -1.27696913, -0.98707581, -1.04956155, -1.11085531,
       -1.78171243, -1.0267196 , -1.10580494, -1.152389  , -1.30279768,
       -1.41076476, -1.06019107, -1.14077871, -1.05265973, -1.14124657,
       -0.9720991 , -1.64791619, -1.02373683, -1.04437468, -1.08591181,
       -1.80068193, -1.04943888, -1.59120386, -0.38580015, -1.32131076,
       -1.05192466, -1.1248968 , -1.25154927, -1.26157156, -1.07650865,
       -1.0632771 , -1.08757241, -1.17701078, -1.05294727, -1.20541104,
       -0.73270748, -1.01614995, -1.13340745, -1.51450216, -1.04640809,
       -1.35970056, -1.05087272, -1.4211699 , -1.09312009, -1.01990633,
       -2.79480857, -1.05470353, -1.61078106, -1.06461134, -1.06071661,
       -1.05871563, -1.52027488, -1.29444009, -1.12961564, -0.88

In [20]:
y_val_inversed[0]

('php', 'sql')

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
def print_evaluation_scores(y_val, predicted):
    
    print ("Accracy={}".format(accuracy_score(y_val, predicted)), 
        "\n\nF1_macro={}".format(f1_score(y_val, predicted, average='macro')),
        "\n\nF1_micro={}".format(f1_score(y_val, predicted, average='micro')),
        "\n\nF1_wted={}".format(f1_score(y_val, predicted, average='weighted')),
        "\n\nPrecsion_macro={}".format(average_precision_score(y_val, predicted, average='macro')),
        "\n\nPrecsion_micro={}".format(average_precision_score(y_val, predicted, average='micro')),
        "\n\nPrecsion_wted={}".format(average_precision_score(y_val, predicted, average='weighted')))

In [0]:
print_evaluation_scores(y_validation, y_val_predicted)


In [0]:
y_val_pred_inversed = mlb.inverse_transform((y_val_predicted[1,:]))


NameError: ignored

In [21]:
import sklearn
sklearn.__version__

'0.21.2'

In [24]:
!pip install  scikit-learn==0.20.1

     |████████████████████████████████| 5.4MB 2.8MB/s 
  Found existing installation: scikit-learn 0.21.2
    Uninstalling scikit-learn-0.21.2:
      Successfully uninstalled scikit-learn-0.21.2
